In [4]:
var = [1, 4, 2, 5, 3]

In [11]:
for i in map(str, var):
    print(i, type(i))

1 <class 'str'>
4 <class 'str'>
2 <class 'str'>
5 <class 'str'>
0 <class 'str'>


In [5]:
var.append(0)
" ".join(map(str, var))

'1 4 2 5 3 0'

In [80]:
def encodage(ne, nj, j, x, y):
    return j * ne**2 + x * ne + y + 1

def decodage(k, ne):
    y = (k - 1) % ne
    x = ((k - 1 - y) // ne) % ne
    j = (k - 1 - y - x * ne) // (ne * ne)
    return j, x, y

print(encodage(3, 4, 1, 0, 1))
print(decodage(11, 3))

11
(1, 0, 1)


In [75]:
var = [1, 4, 2, 5, 3]
def au_moins_un(variables):
    clause = [str(v) for v in variables]
    clause.append('0')
    return [' '.join(clause)]

au_moins_un(var)

['1 4 2 5 3 0']

In [76]:
var = [1, 4, 2, 5, 3]
def au_plus_un(variables):
    clauses = []
    n = len(variables)
    for i in range(n):
        for j in range(i+1, n):
            clauses.append('-{} -{} 0'.format(variables[i], variables[j]))
    return clauses

au_plus_un(var)

['-1 -4 0',
 '-1 -2 0',
 '-1 -5 0',
 '-1 -3 0',
 '-4 -2 0',
 '-4 -5 0',
 '-4 -3 0',
 '-2 -5 0',
 '-2 -3 0',
 '-5 -3 0']

In [77]:
from itertools import combinations
l = [[-i, -j] for i, j in combinations(var, 2)]
print(l)
[" ".join(map(str, clause + [0])) for clause in l]

[[-1, -4], [-1, -2], [-1, -5], [-1, -3], [-4, -2], [-4, -5], [-4, -3], [-2, -5], [-2, -3], [-5, -3]]


['-1 -4 0',
 '-1 -2 0',
 '-1 -5 0',
 '-1 -3 0',
 '-4 -2 0',
 '-4 -5 0',
 '-4 -3 0',
 '-2 -5 0',
 '-2 -3 0',
 '-5 -3 0']

In [78]:
def encoderC1(ne, nj):
    clauses = []
    for j in range(nj):
        for x in range(ne):
            # Contrainte "au plus 1" pour chaque équipe x et chaque jour j
            domicile = [encodage(ne, nj, j, x, y) for y in range(ne) if y != x]
            exterieur = [encodage(ne, nj, j, y, x) for y in range(ne) if y != x]
            clauses.extend(au_plus_un(domicile + exterieur))
    return clauses

encoderC1(3, 4)

['-2 -3 0',
 '-2 -4 0',
 '-2 -7 0',
 '-3 -4 0',
 '-3 -7 0',
 '-4 -7 0',
 '-4 -6 0',
 '-4 -2 0',
 '-4 -8 0',
 '-6 -2 0',
 '-6 -8 0',
 '-2 -8 0',
 '-7 -8 0',
 '-7 -3 0',
 '-7 -6 0',
 '-8 -3 0',
 '-8 -6 0',
 '-3 -6 0',
 '-11 -12 0',
 '-11 -13 0',
 '-11 -16 0',
 '-12 -13 0',
 '-12 -16 0',
 '-13 -16 0',
 '-13 -15 0',
 '-13 -11 0',
 '-13 -17 0',
 '-15 -11 0',
 '-15 -17 0',
 '-11 -17 0',
 '-16 -17 0',
 '-16 -12 0',
 '-16 -15 0',
 '-17 -12 0',
 '-17 -15 0',
 '-12 -15 0',
 '-20 -21 0',
 '-20 -22 0',
 '-20 -25 0',
 '-21 -22 0',
 '-21 -25 0',
 '-22 -25 0',
 '-22 -24 0',
 '-22 -20 0',
 '-22 -26 0',
 '-24 -20 0',
 '-24 -26 0',
 '-20 -26 0',
 '-25 -26 0',
 '-25 -21 0',
 '-25 -24 0',
 '-26 -21 0',
 '-26 -24 0',
 '-21 -24 0',
 '-29 -30 0',
 '-29 -31 0',
 '-29 -34 0',
 '-30 -31 0',
 '-30 -34 0',
 '-31 -34 0',
 '-31 -33 0',
 '-31 -29 0',
 '-31 -35 0',
 '-33 -29 0',
 '-33 -35 0',
 '-29 -35 0',
 '-34 -35 0',
 '-34 -30 0',
 '-34 -33 0',
 '-35 -30 0',
 '-35 -33 0',
 '-30 -33 0']

In [79]:
def encoderC2(ne, nj):
    clauses = []
    for x in range(ne):
            for y in range(ne):
                if x != y:
                    list_match = [encodage(ne, nj, j, x, y) for j in range(nj)]
                    clauses.extend(au_moins_un(list_match))
                    clauses.extend(au_plus_un(list_match))
    return clauses
    
encoderC2(3,4)

['2 11 20 29 0',
 '-2 -11 0',
 '-2 -20 0',
 '-2 -29 0',
 '-11 -20 0',
 '-11 -29 0',
 '-20 -29 0',
 '3 12 21 30 0',
 '-3 -12 0',
 '-3 -21 0',
 '-3 -30 0',
 '-12 -21 0',
 '-12 -30 0',
 '-21 -30 0',
 '4 13 22 31 0',
 '-4 -13 0',
 '-4 -22 0',
 '-4 -31 0',
 '-13 -22 0',
 '-13 -31 0',
 '-22 -31 0',
 '6 15 24 33 0',
 '-6 -15 0',
 '-6 -24 0',
 '-6 -33 0',
 '-15 -24 0',
 '-15 -33 0',
 '-24 -33 0',
 '7 16 25 34 0',
 '-7 -16 0',
 '-7 -25 0',
 '-7 -34 0',
 '-16 -25 0',
 '-16 -34 0',
 '-25 -34 0',
 '8 17 26 35 0',
 '-8 -17 0',
 '-8 -26 0',
 '-8 -35 0',
 '-17 -26 0',
 '-17 -35 0',
 '-26 -35 0']

In [84]:
def encoder(ne, nj):
    return encoderC1(ne, nj) + encoderC2(ne, nj)

ne = 3
nj = 6
with open("contraintes.cnf", "w") as f:
    # Écrire l'en-tête du fichier DIMACS
    f.write("p cnf {} {}\n".format(ne ** 2 * nj, len(encoder(ne, nj))))
    for contrainte in encoder(ne, nj):
        f.write(contrainte + "\n")




In [ ]:
def decoder(sortie):
    # Ouvrir le fichier de sortie de glucose
    with open(sortie, 'r') as f:
        sortie = f.readlines()
    pass